SELECTION DU MODELE

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

: 

In [ ]:

# Download the dataset
dataset = pd.read_csv('dataset_cleaned.csv')

# Display the first few rows of the dataframe
dataset.head()

Preprocessing

In [10]:
import numpy as np
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler



In [ ]:
# 1. Séparation des caractéristiques (features) et de la cible (target)
X = dataset.drop('rendement_moyen', axis=1)
y = dataset['rendement_moyen']

# 2. Identification des colonnes numériques et non numériques
numeric_columns = X.select_dtypes(include=['number']).columns
categorical_columns = X.select_dtypes(exclude=['number']).columns

print("Colonnes numériques :", numeric_columns)
print("Colonnes catégoriques :", categorical_columns)

# 3. Création du préprocesseur
#    - Encodage OneHotEncoder pour les colonnes catégoriques.
#    - Standardisation (StandardScaler) pour les colonnes numériques.
preprocessor = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include=np.number)),  # Standardisation des colonnes numériques
    (OneHotEncoder(), make_column_selector(dtype_exclude=np.number))   # Encodage des colonnes catégoriques
)

# 4. Création du pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# 5. Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Application du pipeline aux données
X_train = pipeline.fit_transform(X_train)  # Ajustement et transformation sur l'ensemble d'entraînement
X_test = pipeline.transform(X_test)        # Transformation uniquement sur l'ensemble de test

RFECV pour Validation Croisée Automatique

In [16]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV



In [ ]:
# Modèle utilisé pour RFECV
model = RandomForestRegressor(random_state=42)

# RFECV pour sélection récursive avec validation croisée
rfecv = RFECV(estimator=model, step=1, cv=5, scoring='r2')  # 5-fold cross-validation
rfecv.fit(X_train, y_train)

# Résultats
print("Nombre optimal de caractéristiques :", rfecv.n_features_)

In [ ]:
mean_cv_scores = rfecv.cv_results_['mean_test_score']
print("Scores moyens (R²) pour chaque nombre de caractéristiques :", mean_cv_scores)


Affichage des Colonnes selectionnées

In [ ]:
# Récupérer les noms des colonnes après transformation
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

# Filtrer les colonnes sélectionnées
selected_features = [feature_names[i] for i in range(len(feature_names)) if rfecv.support_[i]]

# Afficher les colonnes sélectionnées
print("Colonnes sélectionnées par RFECV :")
print(selected_features)
